In [ ]:
import requests
import pandas as pd
import time
from tqdm import tqdm

def normalize_issn(issn):
    """Нормализация ISSN: удаление дефисов и приведение к верхнему регистру"""
    return issn.replace('-', '').upper() if issn else None

# Конфигурация
SCIMAGO_FILE = "/content/scimagojr_2022.csv"
MAX_ARTICLES = 3000
PER_PAGE = 50
API_EMAIL = "SabirovaRA3@ieml.ru"

# 1. Загрузка и подготовка данных Scimago
print("Загрузка данных Scimago...")
try:
    scimago = pd.read_csv(
        SCIMAGO_FILE,
        sep=';',
        encoding='latin1',
        usecols=['Issn', 'SJR Best Quartile']
    ).rename(columns={'SJR Best Quartile': 'quartile'})

    # Создаем словарь ISSN -> Quartile с нормализацией
    issn_quartile = {}
    for idx, row in scimago.dropna(subset=['Issn']).iterrows():
        for issn in str(row['Issn']).split(', '):
            normalized_issn = normalize_issn(issn.strip())
            if normalized_issn and len(normalized_issn) == 8:
                issn_quartile[normalized_issn] = row['quartile']

    print(f"Загружено {len(issn_quartile)} уникальных ISSN из Scimago")

except Exception as e:
    print(f"Ошибка загрузки файла Scimago: {str(e)}")
    raise

# 2. Сбор данных из OpenAlex
print("\nСбор данных из OpenAlex...")

headers = {
    "User-Agent": f"Research Project ({API_EMAIL})",
    "From": API_EMAIL
}

params = {
    "filter": "publication_year:2014-2024,has_doi:true",
    "select": "id,authorships,publication_year,cited_by_count,grants,open_access,primary_location",
    "per-page": PER_PAGE,
    "cursor": "*"
}

data = []
attempts = 0
progress = tqdm(total=MAX_ARTICLES, desc="Сбор статей")

while len(data) < MAX_ARTICLES and attempts < 5:
    try:
        response = requests.get("https://api.openalex.org/works", headers=headers, params=params, timeout=30)
        response.raise_for_status()

        results = response.json().get('results', [])
        if not results:
            print("Нет данных в ответе")
            break

        for work in results:
            try:
                # Извлечение основных данных
                work_id = work['id'].split('/')[-1]
                authorships = work.get('authorships', [])

                # Извлечение ID авторов
                author_ids = []
                for authorship in authorships:
                    author = authorship.get('author', {})
                    if author.get('id'):
                        author_id = author['id'].split('/')[-1]
                        author_ids.append(author_id)

                if not author_ids:  # Пропускаем публикации без авторов
                    continue

                # Обработка журнала
                primary_location = work.get('primary_location', {})
                source = primary_location.get('source', {})
                issn_list = source.get('issn', [])

                # Поиск квартиля
                quartile = None
                for issn in issn_list:
                    normalized_issn = normalize_issn(issn)
                    quartile = issn_quartile.get(normalized_issn)
                    if quartile:
                        break

                if not quartile or quartile not in ['Q1', 'Q2', 'Q3', 'Q4']:
                    continue  # Пропускаем публикации без квартиля

                # Формирование записи
                entry = {
                    "work_id": work_id,
                    "year": work.get('publication_year'),
                    "num_authors": len(author_ids),
                    "author_ids": ",".join(author_ids),
                    "citations": work.get('cited_by_count', 0),
                    "is_oa": work.get('open_access', {}).get('is_oa', False),
                    "has_grant": bool(work.get('grants')),
                    "issn": normalized_issn,
                    "quartile": quartile,
                    "journal_name": source.get('display_name', 'Unknown')
                }

                data.append(entry)
                progress.update(1)

                if len(data) >= MAX_ARTICLES:
                    break

            except Exception as e:
                print(f"\nОшибка обработки статьи: {str(e)}")
                continue

        if 'next_cursor' in response.json().get('meta', {}):
            params['cursor'] = response.json()['meta']['next_cursor']
            time.sleep(1)
        else:
            break

    except Exception as e:
        print(f"\nОшибка запроса: {str(e)}")
        attempts += 1
        time.sleep(10)

progress.close()

# 3. Сохранение результатов
if data:
    df = pd.DataFrame(data)

    print(f"\nСохранено {len(df)} записей. Пример данных:")
    print(df[['work_id', 'author_ids', 'quartile', 'citations']].head())

    print("\nРаспределение квартилей:")
    print(df['quartile'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%')

    # Сохранение основной таблицы
    df.to_csv('publications_with_authors.csv', index=False)

    # Создание таблицы связей автор-публикация
    author_relations = []
    for _, row in df.iterrows():
        for author_id in row['author_ids'].split(','):
            author_relations.append({
                "work_id": row['work_id'],
                "author_id": author_id,
                "quartile": row['quartile']
            })

    pd.DataFrame(author_relations).to_csv('author_publication_mapping.csv', index=False)
    print("\nСоздана таблица связей автор-публикация (author_publication_mapping.csv)")

else:
    print("\nНе удалось собрать данные")

# 4. Валидация данных
if not df.empty:
    print("\nПроверка данных:")
    print(f"Всего статей: {len(df)}")
    print(f"Уникальных авторов: {pd.DataFrame(author_relations)['author_id'].nunique()}")
    print(f"Средняя цитируемость: {df['citations'].mean():.1f}")